In [1]:
%matplotlib notebook
%load_ext autoreload
%autoreload 1
!hostname
!pwd

dv002.bridges2.psc.edu
/ocean/projects/asc170022p/mtragoza/mre-pinn/notebooks


In [2]:
import sys, os, pathlib
import numpy as np
import xarray as xr
import torch
import matplotlib.pyplot as plt
import seaborn as sns

os.environ['DDEBACKEND'] = 'pytorch'
import deepxde

sys.path.append('..')
%aimport mre_pinn

torch.cuda.is_available()

Using backend: pytorch



True

In [3]:
%autoreload

cohort = mre_pinn.data.PatientCohort(xarray_dir='../data/NAFLD', patient_ids='0006')
#cohort.create_xarrays()
#cohort.save_xarrays()
cohort.load_xarrays()

  0%|          | 0/1 [00:00<?, ?it/s]Loading ../data/NAFLD/0006/t1_pre_in.nc
Loading ../data/NAFLD/0006/t1_pre_water.nc
Loading ../data/NAFLD/0006/t1_pre_out.nc
Loading ../data/NAFLD/0006/t1_pre_fat.nc
Loading ../data/NAFLD/0006/t2.nc
Loading ../data/NAFLD/0006/mre_raw.nc
Loading ../data/NAFLD/0006/wave.nc
Loading ../data/NAFLD/0006/mre.nc
Loading ../data/NAFLD/0006/mask.nc
100%|██████████| 1/1 [00:00<00:00, 22.76it/s]


In [4]:
pde = mre_pinn.pde.WaveEquation.from_name('helmholtz', detach=True)
pde

In [5]:
%autoreload
data = mre_pinn.training.PINOData(cohort, pde)
inputs, targets = data.train_next_batch(1)
[x.shape for x in inputs]

[torch.Size([1, 512, 512, 88, 4]),
 torch.Size([1, 512, 512, 88, 3]),
 torch.Size([1, 512, 512, 4, 3])]

In [6]:
%autoreload

pino = mre_pinn.model.PINO(
    n_spatial_dims=3,
    n_channels_in=4,
    n_blocks=0,
    n_hidden=16,
    n_modes=8,
    n_channels_out=1,
    activ_fn='g'
)
pino

PINO(
  (fc_in): Linear(in_features=4, out_features=16, bias=True)
  (conv_out): SpectralConv3d(n_spatial_dims=3, n_channels_in=16, n_channels_out=1, n_modes=8)
)

In [7]:
%autoreload

model = mre_pinn.training.PINOModel(cohort, pino, pde, batch_size=1)
model.compile(
    optimizer='adam',
    lr=1e-4,
    loss_weights=[1e-8, 1e0],
    loss=mre_pinn.training.losses.standardized_msae_loss_fn(np.ones(1))
)

Compiling model...
'compile' took 0.000150 s



In [8]:
model.train(1)

Training model...

torch.Size([1, 512, 512, 4, 1])
torch.Size([])
torch.Size([1, 512, 512, 4, 1])
torch.Size([])
Step      Train loss              Test loss               Test metric
0         [nan, nan]              [nan, nan]              []  
torch.Size([1, 512, 512, 4, 1])
torch.Size([])
torch.Size([1, 512, 512, 4, 1])
torch.Size([])
torch.Size([1, 512, 512, 4, 1])
torch.Size([])
1         [nan, nan]              [nan, nan]              []  

Best model at step 0:
  train loss: inf
  test loss: inf
  test metric: 

'train' took 2.249349 s



(<deepxde.model.LossHistory at 0x147d8f2a6a40>,
 <deepxde.model.TrainState at 0x147d8f2a6380>)